# Final Coursera Project - Melbourne Restaurant Analysis

#### The following notebook contains my final project

Firstly, import the required libraries

In [1]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs"

In [4]:
url_df = pd.read_html(url, header=0)

In [5]:
df = url_df[0]
df.head()

,Suburb,Postcode,Local government area,Location[citation needed],Distance[3][citation needed],Area[citation needed],Population[citation needed],Population density[citation needed],Date established[citation needed]
0,Bellfield,3081,City of Banyule,NaN,NaN,0.9 km2,"1,793[4]",NaN,NaN
1,Briar Hill,3088,City of Banyule,NaN,NaN,NaN,"3,152[4]",NaN,NaN
2,Bundoora,3083,City of Banyule; City of Darebin; City of Whit...,NaN,NaN,15 km2,28653,NaN,NaN
3,Eaglemont,3084,City of Banyule,NaN,NaN,1.9 km2,3873,NaN,NaN
4,Eltham,3095,City of Banyule; Shire of Nillumbik,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df['Address'] = df['Suburb'] + ', VIC ' + df['Postcode'].apply(str) + ", Australia"

In [7]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")

In [8]:
def geoConvLat(address):
    a = geolocator.geocode(address, timeout=15)
    if a != None:
        return a.latitude
    else: return 0
def geoConvLong(address):
    a = geolocator.geocode(address, timeout=15)
    if a != None:
        return a.longitude
    else: return 0

In [9]:
#lats = []
#for i in df['Address']:
#    lats.append(geoConvLat(i))

In [10]:
#longs = []
#for i in df['Address']:
#    longs.append(geoConvLong(i))

In [11]:
#lats = pd.Series(lats)
#longs = pd.Series(longs)
df['Latitude'] = lats
df['Longitude'] = longs

In [12]:
#df.to_csv('suburbs_and_coords.csv')

In [13]:
df = pd.read_csv('suburbs_and_coords.csv')

In [14]:
df[df['Suburb'] == "Woodstock"]

,Unnamed: 0,Suburb,Postcode,Local government area,Location[citation needed],Distance[3][citation needed],Area[citation needed],Population[citation needed],Population density[citation needed],Date established[citation needed],Address,Latitude,Longitude
464,464,Woodstock,3755,City of Whittlesea,NaN,NaN,NaN,NaN,NaN,NaN,"Woodstock, VIC 3755, Australia",-37.546049,145.054427


In [15]:
df.shape

(549, 13)

In [16]:
drop_columns = ['Population[citation needed]', 'Area[citation needed]', 'Population density[citation needed]', 
                'Unnamed: 0', 'Location[citation needed]', 'Distance[3][citation needed]', 'Date established[citation needed]',
               'Address']
df = df.drop(drop_columns, axis=1)

In [17]:
df.head()

,Suburb,Postcode,Local government area,Latitude,Longitude
0,Bellfield,3081,City of Banyule,-37.753107,145.038478
1,Briar Hill,3088,City of Banyule,-37.706370,145.121648
2,Bundoora,3083,City of Banyule; City of Darebin; City of Whit...,-37.697306,145.066254
3,Eaglemont,3084,City of Banyule,-37.765144,145.063331
4,Eltham,3095,City of Banyule; Shire of Nillumbik,-37.713387,145.147880


In [18]:
import folium

# create map using latitude and longitude values
map_melbourne = folium.Map(location=(-37.9136, 144.9631), zoom_start=9)

# add markers to map
for lat, lng, suburb, lga in zip(df['Latitude'], df['Longitude'], df['Suburb'], df['Local government area']):
    label = '{}, {}'.format(suburb, lga)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.5,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

In [19]:
CLIENT_ID = 'LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML' # your Foursquare ID
CLIENT_SECRET = 'AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM' # your Foursquare Secret
VERSION = '20190701' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML
CLIENT_SECRET:AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
lat, long = (-37.9136, 144.9631)
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML&client_secret=AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM&v=20190701&ll=-37.9136,144.9631&radius=1000&limit=100'

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
melbourne_venues = getNearbyVenues(names=df['Suburb'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [23]:
melbourne_venues.groupby('Suburb')['Suburb'].count()
print("Done")

Done


In [24]:
print('There are {} uniques categories.'.format(len(melbourne_venues['Venue Category'].unique())))

There are 218 uniques categories.


In [25]:
# one hot encoding
onehot = pd.get_dummies(melbourne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Suburb'] = melbourne_venues['Suburb'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Suburb,Afghan Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yunnan Restaurant
0,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
onehot.shape

(8113, 219)

In [27]:
grouped = onehot.groupby('Suburb').mean().reset_index()
grouped.head()

,Suburb,Afghan Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Australian Restaurant,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yunnan Restaurant
0,Abbotsford,0.000000,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,...,0.0,0.010000,0.01,0.01,0.0,0.0,0.0,0.000000,0.0,0.01
1,Aberfeldie,0.033333,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.033333,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.00
2,Aintree,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.00
3,Airport West,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.00
4,Albanvale,0.045455,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.045455,0.00,0.00,0.0,0.0,0.0,0.045455,0.0,0.00


In [28]:
grouped.shape

(319, 219)

In [84]:
num_top_venues = 5

for hood in grouped['Suburb'][:2]:
        print("----"+hood+"----")
        temp = grouped[grouped['Suburb'] == hood].T.reset_index()
        temp.columns = ['venue','freq']
        temp = temp.iloc[1:]
        temp['freq'] = temp['freq'].astype(float)
        temp = temp.round({'freq': 10})
        print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')

----Abbotsford----
                 venue  freq
0   Italian Restaurant  0.10
1     Malay Restaurant  0.04
2      Thai Restaurant  0.04
3    Korean Restaurant  0.03
4  Japanese Restaurant  0.02


----Aberfeldie----
                       venue      freq
0          Indian Restaurant  0.100000
1  Middle Eastern Restaurant  0.033333
2  South American Restaurant  0.033333
3         Turkish Restaurant  0.033333
4         Falafel Restaurant  0.033333




In [85]:
keep = ['Suburb']
keep_venues = pd.read_csv("venues.csv")
keep_venues = keep_venues['Venue_Type'] 
keep_venues = list(keep_venues)
for i in keep_venues:
    keep.append(i)

In [86]:
grouped = grouped[keep]

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 1

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Suburb'] = grouped['Suburb']

for ind in np.arange(grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Suburb,1st Most Common Venue
0,Abbotsford,Italian Restaurant
1,Aberfeldie,Indian Restaurant
2,Aintree,Polish Restaurant
3,Airport West,Polish Restaurant
4,Albanvale,Middle Eastern Restaurant


In [89]:
# set number of clusters
#kclusters = 10

#grouped_clustering = grouped.drop('Suburb', 1)

# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_

In [90]:
merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(venues_sorted.set_index('Suburb'), on='Suburb')

merged.head()

,Suburb,Postcode,Local government area,Latitude,Longitude,1st Most Common Venue
0,Bellfield,3081,City of Banyule,-37.753107,145.038478,Middle Eastern Restaurant
1,Briar Hill,3088,City of Banyule,-37.706370,145.121648,Portuguese Restaurant
2,Bundoora,3083,City of Banyule; City of Darebin; City of Whit...,-37.697306,145.066254,Fast Food Restaurant
3,Eaglemont,3084,City of Banyule,-37.765144,145.063331,Indian Restaurant
4,Eltham,3095,City of Banyule; Shire of Nillumbik,-37.713387,145.147880,Polish Restaurant


In [91]:
#merged = merged[np.isfinite(merged['Cluster Labels'])]

In [92]:
merged.to_csv('mergedData.csv')

In [93]:
merged= pd.read_csv('mergedData.csv')

In [94]:
namesList = merged['1st Most Common Venue'].unique()

In [95]:
namesList

array(['Middle Eastern Restaurant', 'Portuguese Restaurant',
       'Fast Food Restaurant', 'Indian Restaurant', 'Polish Restaurant',
       'Thai Restaurant', nan, 'Korean Restaurant', 'Italian Restaurant',
       'Malay Restaurant', 'Chinese Restaurant', 'Tapas Restaurant',
       'Sushi Restaurant', 'Kebab Restaurant', 'Australian Restaurant',
       'Vietnamese Restaurant', 'South American Restaurant',
       'Afghan Restaurant', 'Seafood Restaurant'], dtype=object)

In [97]:
# create map
map_clusters = folium.Map(location=(-37.9136, 144.9631), zoom_start=9)

# set color scheme for the clusters
x = np.arange(0, len(namesList))
ys = [i + x + (i*x)**2 for i in range(len(namesList))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, venue in zip(merged['Latitude'], merged['Longitude'], merged['Suburb'], merged['1st Most Common Venue']):
    label = folium.Popup(str(poi) + ", " + str(venue), parse_html=True)

    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[len(namesList)-1],
        fill=True,
        fill_color=rainbow[len(namesList)-1],
        fill_opacity=1
    ).add_to(map_clusters)
    
       
map_clusters

In [98]:
merged.loc[merged['Cluster Labels'] == 7, merged.columns[[0] + list(range(5, merged.shape[1]))]].iloc[:,:5]

KeyError: 'Cluster Labels'